In spark we can even use complex types and perform something like checking whether a key exists within an
array when you perform a join.

Types of join:
- Inner joins (keep rows with keys that exist in the left and right datasets)
- Outer joins (keep rows with keys in either the left or right datasets)
- Left outer joins (keep rows with keys in the left dataset)
- Right outer joins (keep rows with keys in the right dataset)
- Left semi joins (keep the rows in the left, and only the left, dataset where the key
appears in the right dataset)
- Left anti joins (keep the rows in the left, and only the left, dataset where they do not
appear in the right dataset)
- Natural joins (perform a join by implicitly matching the columns between the two
datasets with the same names)
- Cross (or Cartesian) joins (match every row in the left dataset with every row in the
right dataset)

In [0]:
person = spark.createDataFrame([
(0, "Bill Chambers", 0, [100]),
(1, "Matei Zaharia", 1, [500, 250, 100]),
(2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
(0, "Masters", "School of Information", "UC Berkeley"),
(2, "Masters", "EECS", "UC Berkeley"),
(1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
(500, "Vice President"),
(250, "PMC Member"),
(100, "Contributor")])\
.toDF("id", "status")

In [0]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [0]:
person.show(5)
graduateProgram.show(5)
sparkStatus.show(5)

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



In [0]:
# inner join
joinExpression = person["graduate_program"] == graduateProgram['id']
person.join(graduateProgram,joinExpression, 'inner').show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [0]:
%sql
SELECT * FROM person JOIN graduateProgram
ON person.graduate_program = graduateProgram.id

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chambers,0,List(100),0,Masters,School of Information,UC Berkeley
1,Matei Zaharia,1,"List(500, 250, 100)",1,Ph.D.,EECS,UC Berkeley
2,Michael Armbrust,1,"List(250, 100)",1,Ph.D.,EECS,UC Berkeley


In [0]:
#outer join
joinExpression = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpression, 'outer').show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [0]:
%sql
SELECT * FROM person FULL OUTER JOIN graduateProgram
ON graduate_program = graduateProgram.id

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chambers,0,List(100),0,Masters,School of Information,UC Berkeley
1,Matei Zaharia,1,"List(500, 250, 100)",1,Ph.D.,EECS,UC Berkeley
2,Michael Armbrust,1,"List(250, 100)",1,Ph.D.,EECS,UC Berkeley
null,null,null,null,2,Masters,EECS,UC Berkeley


In [0]:
#left outer join
person.join(graduateProgram, joinExpression, 'left_outer').show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [0]:
#right outer join
person.join(graduateProgram, joinExpression, 'right_outer').show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [0]:
#left semi join
person.join(graduateProgram, joinExpression, 'left_semi').show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [0]:
gradProgram2 = graduateProgram.union(spark.createDataFrame([
(0, "Masters", "Duplicated Row", "Duplicated School")]))

In [0]:
person.join(graduateProgram, joinExpression, 'left_semi').show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [0]:
#left anti joins
graduateProgram.join(person, joinExpression, 'left_anti').show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



In [0]:
#cross join
person.join(graduateProgram, joinExpression, 'cross').show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [0]:
person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  2|Masters|                EECS|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  2|Masters|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  0|Masters|School of Informa...|UC 

In [0]:
sparkStatus.show()
person.show()

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [0]:
from pyspark.sql.functions import expr, array_contains
person.withColumnRenamed('id','personID').join(sparkStatus, array_contains(person["spark_status"],sparkStatus["id"])).show()

+--------+----------------+----------------+---------------+---+--------------+
|personID|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



Duplication while using join can occur in two distinct situations:
- The join expression that you specify does not remove one key from one of the input
DataFrames and the keys have the same column name
- Two columns on which you are not performing the join have the same name

In [0]:
joinExpression

Out[18]: Column<'(graduate_program = id)'>

In [0]:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")
person.join(gradProgramDupe, person['graduate_program'] == gradProgramDupe['graduate_program']).show()

+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|graduate_program| degree|          department|     school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|               0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+



In [0]:
person.join(gradProgramDupe, person['graduate_program'] == gradProgramDupe['graduate_program']).select("graduate_program").show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-845529134050329> in <cell line: 1>()
----> 1 person.join(gradProgramDupe, person['graduate_program'] == gradProgramDupe['graduate_program']).select("graduate_program").show()

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/dataframe.py in select(self, *cols)
   2107         [Row(name='Alice', age=12), Row(name='Bob', age=15)]
   2108         """
-> 2109         jdf = self._jdf.select(self._jcols(*cols))
   2110         return DataFrame(jdf, self.sparkSession)
   2111 

/databricks/

In [0]:
#solution 1 for duplicates
person.join(gradProgramDupe, 'graduate_program').show()

+----------------+---+----------------+---------------+-------+--------------------+-----------+
|graduate_program| id|            name|   spark_status| degree|          department|     school|
+----------------+---+----------------+---------------+-------+--------------------+-----------+
|               0|  0|   Bill Chambers|          [100]|Masters|School of Informa...|UC Berkeley|
|               1|  1|   Matei Zaharia|[500, 250, 100]|  Ph.D.|                EECS|UC Berkeley|
|               1|  2|Michael Armbrust|     [250, 100]|  Ph.D.|                EECS|UC Berkeley|
+----------------+---+----------------+---------------+-------+--------------------+-----------+



In [0]:
a = person.join(gradProgramDupe, 'graduate_program')
a.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [graduate_program#20L, id#18L, name#19, spark_status#21, degree#35, department#36, school#37]
   +- SortMergeJoin [graduate_program#20L], [graduate_program#691L], Inner
      :- Sort [graduate_program#20L ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(graduate_program#20L, 200), ENSURE_REQUIREMENTS, [plan_id=2454]
      :     +- Project [_1#10L AS id#18L, _2#11 AS name#19, _3#12L AS graduate_program#20L, _4#13 AS spark_status#21]
      :        +- Filter isnotnull(_3#12L)
      :           +- Scan ExistingRDD[_1#10L,_2#11,_3#12L,_4#13]
      +- Sort [graduate_program#691L ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(graduate_program#691L, 200), ENSURE_REQUIREMENTS, [plan_id=2455]
            +- Project [_1#26L AS graduate_program#691L, _2#27 AS degree#35, _3#28 AS department#36, _4#29 AS school#37]
               +- Filter isnotnull(_1#26L)
                  +- Scan ExistingRDD[_1#2

In [0]:
#solution 2 for duplicates
person.join(gradProgramDupe, person['graduate_program'] == gradProgramDupe['graduate_program'])\
.drop(person["graduate_program"]).show()

+---+----------------+---------------+----------------+-------+--------------------+-----------+
| id|            name|   spark_status|graduate_program| degree|          department|     school|
+---+----------------+---------------+----------------+-------+--------------------+-----------+
|  0|   Bill Chambers|          [100]|               0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|[500, 250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|     [250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+---------------+----------------+-------+--------------------+-----------+



In [0]:
#solution 3 for duplicates: rename columns before join
from pyspark.sql.functions import col
gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
joinExpr = person["graduate_program"] == gradProgram3["grad_id"]
person.join(gradProgram3, joinExpr).show()

+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|grad_id| degree|          department|     school|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|      0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|      1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|      1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+



In [0]:
#few ways to add suffix:
'''df1_r = df1.select(*(col(x).alias(x + '_df1') for x in df1.columns))
df2_r = df2.select(*(col(x).alias(x + '_df2') for x in df2.columns))'''

#approach 2
#instead of alias, to access a specific col, use parent dataframe name like parent["col"]

two core resources at play:
- the node-to-node communication strategy and per node computation strategy

Spark approaches cluster communication in two different ways during joins:
- shuffle join, which results in an all-to-all communication, When you join a big table to another big table, you end up with a shuffle join

In a shuffle join, every node talks to every other node and they share data according to which
node has a certain key or set of keys (on which you are joining). These joins are expensive
because the network can become congested with traffic, especially if your data is not partitioned
well.
- broadcast join

Big table–to–small table
When the table is small enough to fit into the memory of a single worker node, with some
breathing room of course, we can optimize our join. Although we can use a big table–to–big
table communication strategy, it can often be more efficient to use a broadcast join.

With the DataFrame API, we can also explicitly give the optimizer a hint that we would like to
use a broadcast join by using the correct function around the small DataFrame in question.

Note **if you try to broadcast something too large, you can crash your
driver node**

It is important to consider is if you partition your data correctly prior to a join,
you can end up with much more efficient execution because even if a shuffle is planned, if data
from two different DataFrames is already located on the same machine, Spark can avoid the
shuffle. Experiment with some of your data and try partitioning beforehand to see if you can
notice the increase in speed when performing those joins.

In [0]:
from pyspark.sql.functions import broadcast
joinExpr = person["graduate_program"] == graduateProgram["id"]
person.join(broadcast(graduateProgram), joinExpr).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [graduate_program#20L], [id#34L], Inner, BuildRight, false
   :- Project [_1#10L AS id#18L, _2#11 AS name#19, _3#12L AS graduate_program#20L, _4#13 AS spark_status#21]
   :  +- Filter isnotnull(_3#12L)
   :     +- Scan ExistingRDD[_1#10L,_2#11,_3#12L,_4#13]
   +- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=2538]
      +- Project [_1#26L AS id#34L, _2#27 AS degree#35, _3#28 AS department#36, _4#29 AS school#37]
         +- Filter isnotnull(_1#26L)
            +- Scan ExistingRDD[_1#26L,_2#27,_3#28,_4#29]




IN SQL, You can set one of these hints
by using a special comment syntax. MAPJOIN, BROADCAST, and BROADCASTJOIN all do the same
thing and are all supported:

In [0]:
%sql
SELECT /*+ MAPJOIN(graduateProgram) */ * FROM person JOIN graduateProgram
ON person.graduate_program = graduateProgram.id

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chambers,0,List(100),0,Masters,School of Information,UC Berkeley
1,Matei Zaharia,1,"List(500, 250, 100)",1,Ph.D.,EECS,UC Berkeley
2,Michael Armbrust,1,"List(250, 100)",1,Ph.D.,EECS,UC Berkeley
